- https://github.com/facebookresearch/dinov2
- https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/DINOv2/Train_a_linear_classifier_on_top_of_DINOv2_for_semantic_segmentation.ipynb#scrollTo=rLzR_mt_SnE2
- https://github.com/NielsRogge/Transformers-Tutorials/blob/master/DINOv2/Fine_tune_DINOv2_for_image_classification_%5Bminimal%5D.ipynb
- 

- dinov2 has 14x14 patch size. so (floor(width)x2 ) +1 hidden

In [ ]:
# import torch
# from transformers import pipeline

# pipe = pipeline(
#     task="image-classification",
#     model="facebook/dinov2-small-imagenet1k-1-layer",
#     dtype=torch.float16,
#     device=0
# )

# pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg")

In [ ]:
import requests
from transformers import TorchAoConfig, AutoImageProcessor, AutoModelForImageClassification
#from torchao.quantization import Int4WeightOnlyConfig
from PIL import Image

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

import matplotlib.pyplot as plt 

plt.imshow(image)

In [ ]:
import os
import pandas as pd
from torchvision.io import decode_image
import torch
from torch.utils.data import Dataset

from torch.utils.data import Dataset
from PIL import Image
import os
import albumentations as A
import cv2 
import numpy as np 


class SegmentationDataset(Dataset):
    def __init__(self, images, masks, image_dir, mask_dir, transform=None):
        self.images = images
        self.masks = masks 
        self.transform = transform
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        # print(self.image_list)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])  
        image = Image.open(img_path).convert("RGB")
        original_image = np.array(image)
        original_mask = np.load(mask_path)

        transformed = self.transform(image=original_image, mask=original_mask)
        image, target = image, target = torch.tensor(transformed['image']), torch.LongTensor(transformed['mask'])
        image = image.permute(2,0,1)
        return image, target, original_image, original_mask # , original_image, original_segmentation_map
        # if self.transform:
        #     augmented = self.transform(image=np.array(image), mask=np.array(mask))
        #     image, mask = augmented["image"], augmented["mask"]
        # else:
        #     image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
        #     mask = torch.from_numpy(mask).float()

        # image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
        # mask = torch.from_numpy(mask).float()
        
        return image, mask

img_dir = '/mnt/gis/image/18/'
mask_dir = '/mnt/gis/label/18/'
images = os.listdir(img_dir)
masks = os.listdir(mask_dir)


coords = []
for mask in masks:
    x,y = mask.split('_')
    x,y = int(x), int(y.replace('.npy', ''))
    coords.append((x,y))

images = [f'18_{x}_{y}.jpg' for (x,y) in coords]

from sklearn.model_selection import train_test_split



config = {
        'batch_size': 32,
        'epochs': 5,
        'learning_rate': 1e-4,
        'val_split': 0.2,
        'num_workers': 4,
    }


train_images, val_images, train_masks, val_masks = train_test_split(
    images, masks, test_size=config['val_split'], random_state=42
)

TARGET_SIZE = (256, 256) 

import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

width = 256 

train_transform = A.Compose([
    # hadded an issue with an image being too small to crop, PadIfNeeded didn't help...
    # if anyone knows why this is happening I'm happy to read why
    # A.PadIfNeeded(min_height=448, min_width=448),
    # A.RandomResizedCrop(height=448, width=448),
    A.Resize(width=width, height=width),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),
])

val_transform = A.Compose([
    A.Resize(width=width, height=width),
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),

])
train_dataset = SegmentationDataset(train_images, train_masks, image_dir=img_dir, mask_dir=mask_dir, transform=train_transform)#, transform=transform)
val_dataset = SegmentationDataset(val_images, val_masks, image_dir=img_dir, mask_dir=mask_dir, transform=val_transform)#,  transform=transform)

from torch.utils.data import DataLoader

train_loader =  DataLoader(train_dataset, batch_size=8)
val_loader =  DataLoader(val_dataset, batch_size=8)


for images, masks, o,m in train_loader:
    print(images)
    images, masks = images.cuda(), masks.cuda()
    break


In [ ]:
id2label = {
    0: "other",
    1: "track",
}


import torch
from transformers import Dinov2Model, Dinov2PreTrainedModel
from transformers.modeling_outputs import SemanticSegmenterOutput




class Dinov2ForSemanticSegmentation(Dinov2PreTrainedModel):
  def __init__(self, config):
    super().__init__(config)

    self.dinov2 = Dinov2Model(config)
    #self.classifier = LinearClassifier(config.hidden_size, 32, 32, config.num_labels)

  def forward(self, pixel_values, output_hidden_states=False, output_attentions=False, labels=None):
    # use frozen features
    outputs = self.dinov2(pixel_values,
                            output_hidden_states=output_hidden_states,
                            output_attentions=output_attentions)
    # get the patch embeddings - so we exclude the CLS token
    patch_embeddings = outputs.last_hidden_state[:,1:,:]
    return patch_embeddings

  
model = Dinov2ForSemanticSegmentation.from_pretrained("facebook/dinov2-base", id2label=id2label, num_labels=len(id2label))

In [ ]:
for name, param in model.named_parameters():
    if name.startswith("dinov2"):
        param.requires_grad = False

In [ ]:
batch = next(iter(train_loader))
batch_image, batch_label, batch_original_image, bm = batch
batch_image.shape, batch_label.shape

In [ ]:
import matplotlib.pyplot as plt 

def check_dataset(dataset, n=5):
    for i in range(n):
        sample_image, sample_mask, sample_oimage, _ = dataset[i]
        #print(sample_image)
        # print(sample_image.shape, sample_mask.shape)
        fig, axs = plt.subplots(1,3,figsize=(15,5))
        axs[0].imshow(sample_oimage)
        axs[1].imshow(sample_image.permute(1, 2, 0))
        axs[2].imshow(sample_mask)
        plt.show()

check_dataset(train_dataset)

In [ ]:
outputs = model(pixel_values=batch_image, labels=batch_label)
outputs.shape

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import cv2
from pathlib import Path


for i in range(8):
    features = outputs[i].numpy()
    feature_dim = 768

    patches = features.reshape(-1, feature_dim)
    print(patches.shape)

    scaler = StandardScaler()
    patches_scaled = scaler.fit_transform(patches)

    # Apply PCA to reduce to 3 components (RGB)
    pca = PCA(n_components=3, random_state=42)
    pca_result = pca.fit_transform(patches_scaled) 
    pca_result.shape

    pca_min = pca_result.min(axis=0)
    pca_max = pca_result.max(axis=0)
    pca_normalized = (pca_result - pca_min) / (pca_max - pca_min + 1e-8)


    pca_spatial = pca_normalized.reshape(18, 18, 3)
    pca_spatial.shape

    fig, axs = plt.subplots(1,3,figsize=(15,5))
    axs[0].imshow(batch_image[i].permute(1,2,0))
    axs[1].imshow(pca_spatial)
    axs[2].imshow(batch_original_image[i])
    plt.show()